## Working model for tensorfusion

In [4]:
import pickle
import torch
from torchmetrics import F1Score
import pandas as pd
print(torch.__version__)


/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [5]:
# data load
# Raw dataset의 key는 ['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence']로 구성

with open('../model/data/paradeigma_dataset_1_3.pkl', 'rb') as f:
    raw_dataset = pickle.load(f)

# with open('../model/data/paradeigma_dataset_1_4.pkl', 'rb') as f:
#     raw_dataset = pickle.load(f)
    
sessions = ['Session01', 'Session02']
session = sessions[0]
raw_dataset[session].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])

In [6]:
# 40개 세션의 감정 종류는 24개

from glob import glob
annot_lst = glob('../org_KEMDy20/annotation/*.csv')
emotion_list = []
for annot_file in annot_lst:
    annot = pd.read_csv(annot_file, skiprows=1)
    emotion_list.append(annot['Emotion'])
emotion_list = list(pd.Series([j for i in emotion_list for j in i]).unique())
len(emotion_list)

24

In [7]:
raw_dataset['Session01']['Emotion']

0      neutral
1      neutral
2      neutral
3      neutral
4      neutral
        ...   
306    neutral
307    neutral
308    neutral
309    neutral
310    neutral
Name: Emotion, Length: 311, dtype: object

In [8]:
# 세션 1,2 의 감정 종류는 7개

emotion_lst = []
sessions = ['Session01', 'Session02']
for session in sessions:
    emotion_lst.append(raw_dataset[session]['Emotion'].unique())
emotion_lst = [j for i in emotion_lst for j in i]
emotion_lst = list(pd.Series(emotion_lst).unique())
len(emotion_lst)

7

In [9]:
# emotion target에 대한 정수 인코딩, 디코딩

encode_dict = {b:i for i, b in enumerate(emotion_list)}
decode_dict = {i:b for i, b in enumerate(emotion_list)}
encode_dict, decode_dict

({'happy': 0,
  'neutral': 1,
  'surprise': 2,
  'surprise;neutral': 3,
  'sad': 4,
  'neutral;sad': 5,
  'happy;neutral': 6,
  'angry;neutral': 7,
  'neutral;disqust': 8,
  'angry': 9,
  'fear': 10,
  'neutral;fear': 11,
  'disqust': 12,
  'happy;surprise': 13,
  'happy;angry;neutral': 14,
  'angry;disqust': 15,
  'happy;surprise;neutral': 16,
  'happy;fear': 17,
  'happy;neutral;fear': 18,
  'angry;neutral;disqust': 19,
  'neutral;disqust;sad': 20,
  'angry;neutral;disqust;fear;sad': 21,
  'happy;sad': 22,
  'happy;neutral;disqust': 23},
 {0: 'happy',
  1: 'neutral',
  2: 'surprise',
  3: 'surprise;neutral',
  4: 'sad',
  5: 'neutral;sad',
  6: 'happy;neutral',
  7: 'angry;neutral',
  8: 'neutral;disqust',
  9: 'angry',
  10: 'fear',
  11: 'neutral;fear',
  12: 'disqust',
  13: 'happy;surprise',
  14: 'happy;angry;neutral',
  15: 'angry;disqust',
  16: 'happy;surprise;neutral',
  17: 'happy;fear',
  18: 'happy;neutral;fear',
  19: 'angry;neutral;disqust',
  20: 'neutral;disqust;sad',

In [10]:
raw_dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])

In [11]:
## dataset을 합치는 과정
## 다시 리뷰해야 할 것

merged_dataset = {}
for session_key in raw_dataset.keys():
    raw_dataset[session_key]['Emotion'] = raw_dataset[session_key]['Emotion'].map(encode_dict)
    for data_name in raw_dataset[session_key].keys():
        if data_name in merged_dataset.keys():
            for data in raw_dataset[session_key][data_name]:
                merged_dataset[data_name].append(data)
        else:
            merged_dataset[data_name] = []
            for data in raw_dataset[session_key][data_name]:
                merged_dataset[data_name].append(data)
    
for data_name in merged_dataset.keys():
    if data_name == 'text_embeddings' or data_name == 'wav_embeddings':
        merged_dataset[data_name] = torch.stack(merged_dataset[data_name])

merged_dataset.keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])

In [12]:
from collections import Counter

sorted(Counter(merged_dataset['Emotion']))

[0, 1, 2, 3, 6, 7, 13]

In [13]:
for i in merged_dataset.keys():
    print(f"{i}: ", len(merged_dataset[i]))

file_names:  576
text_embeddings:  576
wav_embeddings:  576
Emotion:  576
Arousal:  576
Valence:  576


## Torch dataset 만들기


In [14]:
import os 
import pandas as pd
from datasets import Dataset
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split

In [15]:
# data load 및 나누기: https://076923.github.io/posts/Python-pytorch-11/

class EtriDataset(Dataset):
    def __init__(self, file_names, text_embeddings, wave_embeddings, Emotion, Arousal, Valence):
        self.file_names = file_names
        self.text_embeddings = text_embeddings
        self.wav_embeddings = wave_embeddings
        self.label_emotion = Emotion
        self.label_arousal = Arousal
        self.label_valence = Valence
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        text_embeddings = self.text_embeddings[idx]
        wav_embeddings = self.wav_embeddings[idx]
        label_emotion = self.label_emotion[idx]
        label_arousal = self.label_arousal[idx]
        label_valence = self.label_valence[idx]
        return text_embeddings, wav_embeddings, label_emotion, label_arousal, label_valence

In [16]:
# 개별 session 데이터 셋을 만들었을 때 
# dataset = EtriDataset(raw_dataset[session]['file_names'],
#                       raw_dataset[session]['text_embeddings'],
#                       raw_dataset[session]['wav_embeddings'],
#                       raw_dataset[session]['Emotion'],
#                       raw_dataset[session]['Arousal'],
#                       raw_dataset[session]['Valence'])

# session을 통합시킨 데이터 셋을 만들었을 때 
dataset = EtriDataset(merged_dataset['file_names'],
                      merged_dataset['text_embeddings'],
                      merged_dataset['wav_embeddings'],                     
                      merged_dataset['Emotion'],
                      merged_dataset['Arousal'],
                      merged_dataset['Valence'])

In [17]:
dataset_size = len(dataset)
train_size = int(dataset_size * 0.7)
validation_size = int(dataset_size * 0.15)
test_size = dataset_size- (train_size + validation_size)

In [18]:
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

print(train_size, test_size, validation_size)
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

403 87 86
Training Data Size : 403
Validation Data Size : 86
Testing Data Size : 87


In [19]:
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle= True, drop_last = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 32, shuffle= True, drop_last = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle= True, drop_last = True)

## NetWork 만들기

In [20]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
print(f'Using {device} device')

Using cuda device


In [21]:
class MLPNetwork_pre(nn.Module):
    def __init__(self, input_length, input_width):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width, 768)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(768)
        self.fc2 = nn.Linear(768, 512)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512,32)
        self.gelu3 = nn.GELU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        output = self.gelu3(x)
        return output
    
class MLPNetwork_final(nn.Module):
    def __init__(self, input_length, input_width):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width, 256)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 2)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        output = self.fc3(x)
        
        return output

In [22]:
class TensorFusionMixer(nn.Module):
    def __init__(self, ModelA, ModelB):
        super().__init__()
        self.ModelA = ModelA
        self.ModelB = ModelB
        self.Model_mlp_final = MLPNetwork_final(32, 32).to(device)
        self.softmax = nn.Softmax(dim=1)
        
    def tensor_fusion(self, batch_arr1, batch_arr2):
        fusion_matrix_lst = []
        for i, (arr1, arr2) in enumerate(zip(batch_arr1, batch_arr2)):
            outer_matrix = torch.outer(arr1, arr2)
            l, w = outer_matrix.shape
            outer_matrix = outer_matrix.view(1, l, w)
            fusion_matrix_lst.append(outer_matrix)
        fusion_matrix = torch.concat(fusion_matrix_lst)
        
        return fusion_matrix
    
    def forward(self, x1, x2):
        x1 = self.ModelA(x1)
        x2 = self.ModelB(x2)
        fusion_matrix = self.tensor_fusion(x1, x2)
        x = self.Model_mlp_final(fusion_matrix)
        
        return x

In [23]:
txt_input_length, txt_input_width = merged_dataset['text_embeddings'][0].shape
_, wav_input_length, wav_input_width = merged_dataset['wav_embeddings'][0].shape

model_mlp_txt = MLPNetwork_pre(txt_input_length,txt_input_width).to(device)
model_mlp_wav = MLPNetwork_pre(wav_input_length,wav_input_width).to(device)

model_tf_mixer = TensorFusionMixer(ModelA = model_mlp_txt, ModelB = model_mlp_wav).to(device)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_mlp_txt = nn.DataParallel(model_mlp_txt).to(device)
    model_mlp_wav = nn.DataParallel(model_mlp_wav).to(device)
    model_tf_mixer = nn.DataParallel(model_tf_mixer).to(device)

print(model_tf_mixer)

Let's use 4 GPUs!
DataParallel(
  (module): TensorFusionMixer(
    (ModelA): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=61440, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=True)
      (gelu2): GELU(approximate='none')
      (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc3): Linear(in_features=512, out_features=32, bias=True)
      (gelu3): GELU(approximate='none')
    )
    (ModelB): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=37632, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=Tr

In [24]:
class CCC(nn.Module):
    def __init__(self):
        super(CCC, self).__init__()
        self.mean = torch.mean
        self.var = torch.var
        self.sum = torch.sum
        self.sqrt = torch.sqrt
        self.std = torch.std
        
    def forward(self, pred, target):
        mean_gt = self.mean (target, 0)
        mean_pred = self.mean (pred, 0)
        var_gt = self.var (target, 0)
        var_pred = self.var (pred, 0)
        v_pred = pred - mean_pred
        v_gt = target - mean_gt
        cor = self.sum (v_pred * v_gt) / (self.sqrt(self.sum(v_pred ** 2)) * self.sqrt(self.sum(v_gt ** 2)))
        sd_gt = self.std(target)
        sd_pred = self.std(pred)
        numerator = 2 * cor * sd_gt * sd_pred
        denominator = var_gt + var_pred + (mean_gt-mean_pred) ** 2
        ccc = numerator / denominator
        return ccc
    
ccc = CCC()

In [25]:
ccc(torch.tensor([1.0,2.0,3.0,4.0,5.0]),torch.tensor([6.0,7.0,8.0,9.0,10.0]))

tensor(0.1667)

In [ ]:
 # 1번
        # 128,1 / 128, 1
        pred_ccc_a = ccc(pred_a, y_a)
        pred_ccc_v= ccc(pred_v, y_v)
        # : 너희들 arousal valence -가지고 있는 관계성을 맞춰봐 ([1,2,3,.....,128],[1,2,3,.....128])
        # : 평균 표준편차 의미가 있는 값?
        
        # 2번
        # 128,1 / 128, 1
        # ccc(pred_a, y_a) = 1
        # ccc(pred_v, y_v) = 1
        # : 너희들 예측 a 실제 a 관계를 1
        # : 너희들 예측 v 실제 v 관계를 1 
        # : accuracy 대체 
        
        # 3번
        # : 우리가 예측한 a, v와 실제 a, v가 가지고 있는 관계성을 보여봐 ([0.0, 0.0],[0.0, 0.0]) 
        # : 1이 되기를 바람? 
        # 2,1 / 2,1
        # ccc([pred_a , pred_v], [target_a, target_v])
        # : ccc : 하나의 리스트의 평균이랑 표준편차 / 피어슨 상관계수 

In [39]:
# data 순서: text_embeddings, wav_embeddings, label_emotion, label_arousal, label_valence

def train(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    for batch, (X_txt, X_wav, _, y_a, y_v) in enumerate(dataloader):

        X_txt, X_wav, y_a, y_v = X_txt.to(device), X_wav.to(device), y_a.type(torch.FloatTensor).to(device), y_v.type(torch.FloatTensor).to(device)
        pred = model(X_txt, X_wav)
        # 128,2
        
        pred_a = pred[:,0]
        pred_v = pred[:,1]
        
        loss_a = loss_fn(pred_a, y_a)
        loss_v = loss_fn(pred_v, y_v)
        
        pred_ccc_a = ccc(pred_a, y_a)
        pred_ccc_v= ccc(pred_v, y_v)
        
        ccc_mean = (pred_ccc_a + pred_ccc_v) / 2

        optimizer.zero_grad()
        loss_a.backward(retain_graph = True)
        loss_v.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss_a, loss_v, current = loss_a.item(), loss_v.item(), batch * len(X_txt)
            print(f"loss_a: {loss_a:>7f}, loss_b: {loss_v:>7f},  [{current:>5d}/{size:>5d}]")
            print(f"ccc_mean : {ccc_mean:>9f}, Arousal_ccc : {pred_ccc_a:>9f}, Valence_ccc : {pred_ccc_v:>9f}")

In [40]:
def test(dataloader, model, loss_fn, mode = 'test'):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss_a = 0
    test_loss_v = 0
    ccc_mean = 0
    ccc_a = 0
    ccc_v = 0
    
    preds = []
    targets = []
    with torch.no_grad():
        for batch, (X_txt, X_wav, _, y_a, y_v) in enumerate(dataloader):

            X_txt, X_wav, y_a, y_v = X_txt.to(device), X_wav.to(device), y_a.type(torch.FloatTensor).to(device), y_v.type(torch.FloatTensor).to(device)
            pred = model(X_txt, X_wav)
            
            pred_a = pred[:,0]
            pred_v = pred[:,1]
            
            preds.append([pred_a, pred_v])
            targets.append([y_a, y_v])
            
            pred_ccc_a = ccc(pred_a, y_a)
            pred_ccc_v = ccc(pred_v, y_v)
            
            ccc_a += pred_ccc_a
            ccc_v += pred_ccc_v
            ccc_mean += (pred_ccc_a + pred_ccc_v) / 2
            
            test_loss_a += loss_fn(pred_a, y_a).item()
            test_loss_v += loss_fn(pred_v, y_v).item()
            
    test_loss_a /= num_batches
    test_loss_v /= num_batches
    ccc_mean /= num_batches
    ccc_a /= num_batches
    ccc_v /= num_batches

    if mode == 'test':
        print(torch.cat(preds), torch.cat(preds).shape)
        print(f"loss_a: {test_loss_a:>7f}, loss_v: {test_loss_v:>7f}")
        print(f"Tess ccc_mean : {ccc_mean:>9f}, Arousal_ccc : {ccc_a:>9f}, Valence_ccc{ccc_v:>9f}")
        
    elif mode == 'val':
        print(f"loss_a: {test_loss_a:>7f}, loss_v: {test_loss_v:>7f}")
        print(f"Validation ccc_mean : {ccc_mean:>9f}, Arousal_ccc : {ccc_a:>9f}, Valence_ccc{ccc_v:>9f}")    
 
                    

In [41]:
from tqdm import tqdm
lr = 1e-3
loss_fn = nn.MSELoss().to(device)
optimizer = optim.SGD(model_tf_mixer.parameters(), lr=lr)

epochs = 100
for epoch in tqdm(range(epochs)):
    print(f"---------------Epoch {epoch+1}----------------")
    print("Training......")
    train(train_dataloader, model_tf_mixer, loss_fn, optimizer)
    print("Test or Validation......")
    test(validation_dataloader, model_tf_mixer, loss_fn, mode = 'val')
print("Done!")

  0%|          | 0/100 [00:00<?, ?it/s]

---------------Epoch 1----------------
Training......
loss_a: 7.687414, loss_b: 8.272012,  [    0/  403]
ccc_mean : 0.000001, Arousal_ccc : 0.000001, Valence_ccc : 0.000000
Test or Validation......


  1%|          | 1/100 [00:38<1:03:55, 38.75s/it]

loss_a: 7.774595, loss_v: 8.763523
Validation ccc_mean :  0.000001, Arousal_ccc :  0.000001, Valence_ccc-0.000000
---------------Epoch 2----------------
Training......
loss_a: 7.497075, loss_b: 8.427500,  [    0/  403]
ccc_mean : 0.000001, Arousal_ccc : 0.000001, Valence_ccc : 0.000000
Test or Validation......


  2%|▏         | 2/100 [01:14<1:00:33, 37.07s/it]

...

loss_a: 2.457895, loss_v: 2.595644
Validation ccc_mean :  0.000002, Arousal_ccc :  0.000004, Valence_ccc-0.000001
---------------Epoch 40----------------
Training......
loss_a: 2.509080, loss_b: 2.545574,  [    0/  403]
ccc_mean : 0.000003, Arousal_ccc : 0.000005, Valence_ccc : 0.000001
Test or Validation......


 40%|████      | 40/100 [24:37<36:47, 36.79s/it]

loss_a: 2.400854, loss_v: 2.619709
Validation ccc_mean :  0.000002, Arousal_ccc :  0.000004, Valence_ccc-0.000000
---------------Epoch 41----------------
Training......


 40%|████      | 40/100 [24:43<37:04, 37.08s/it]


KeyboardInterrupt: 